# INFO8010: Project
Detailed description: see [README.md](README.md)

## 0. Imports

Importing torch and required libraries

In [ ]:
from matplotlib.pyplot import plt
import numpy as np
import torch
import torch.nn as nn

## 0'. Global constants

In [1]:
SAMPLING_RATE = 12000 # Hz (!= GTZAN dataset sampling rate)

## 1. Data loading

First, define the loader to be used to load the data for the GTZAN dataset (to start with, to be patched if not sufficient)

## 1. Defining the encoder and the decoder

First define as `nn.Module`s:
- the encoder
   - Input: waveform
   - Output: latent representation
- the decoder
   - Input: latent representation
   - Ouptput: waveform
- an encoder-decoder that chains them both for convenience

In [ ]:
# number of samples between successive frames
mel_hop_length = 256
mel_n_mels = 128

class Encoder(nn.Module):
    def __init__(self, n_inputs):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(n_inputs, 123),
            nn.ReLU(),
            nn.Linear(123, 3)
        )
    
    def forward(self, x):
        ms = librosa.feature.melspectrogram(
            x, sr=SAMPLING_RATE,
            hop_length=mel_hop_length,
            window="hann",
            n_mels=)
        return self.encoder(x)


class Decoder(nn.Module):
    def __init__(self, n_outputs):
        super().__init__()

        self.decoder = nn.Sequential(
            nn.Linear(3, 123),
            nn.ReLU(),
            nn.Linear(123, n_outputs)
        )
    
    def forward(self, x):
        return self.decoder(x)

class EncoderDecoder(nn.Module):
    def __init__(self, n_inputs):
        super().__init__()

        self.encoder = Encoder(n_inputs)
        self.decoder = Decoder(n_inputs)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x